In [ ]:
import xarray as xr
import numpy as np
from datetime import datetime as dt

from metpy.units import units
from metpy import calc as mpcalc
from ecape.calc import calc_ecape

In [ ]:
date_i_want = dt(2022, 6, 2)
tfm_path = f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/seabreeze-obs.zarr'
tfm = xr.open_dataset(tfm_path, engine='zarr')

In [ ]:
feature_pressure_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))
feature_msl_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))
feature_temp_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))
feature_dew_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))
feature_u_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))
feature_v_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))
feature_ccn_profile = np.zeros((tfm.feature.shape[0], tfm.vertical_levels.shape[0]))

feature_mlcape = np.zeros((tfm.feature.shape[0]))
feature_mlcin = np.zeros((tfm.feature.shape[0]))
feature_mlecape = np.zeros((tfm.feature.shape[0]))

In [4]:
for sidenum, side in enumerate(['continental', 'maritime']):
    sidenum -= 2
    temp = tfm[f'{side}_temperature_profile'].data * units.degC
    dew = tfm[f'{side}_dewpoint_profile'].data * units.degC
    pressure = tfm[f'{side}_pressure_profile'].data * units.hPa
    height = tfm[f'{side}_msl_profile'].data * units.m
    u = tfm[f'{side}_u_profile'].data * (units.m/units.s)
    v = tfm[f'{side}_v_profile'].data * (units.m/units.s)
    ccn = tfm[f'{side}_ccn_profile'].data

    mlcapes = np.zeros(tfm.time.shape[0])
    mlcins = np.zeros(tfm.time.shape[0])
    mlecapes = np.zeros(tfm.time.shape[0])
    for i in range(tfm.time.shape[0]):
        temp_i = temp[i, :]
        dew_i = dew[i, :]
        pressure_i = pressure[i, :]
        height_i = height[i, :]
        u_i = u[i, :]
        v_i = v[i, :]

        mlcape, mlcin = mpcalc.mixed_layer_cape_cin(pressure_i, temp_i, dew_i)
        spc_hum = mpcalc.specific_humidity_from_dewpoint(pressure_i, dew_i)
        mlecape = calc_ecape(height_i, pressure_i, temp_i, spc_hum, u_i, v_i, cape_type='mixed_layer')

        mlcapes[i] = mlcape.magnitude
        mlcins[i] = mlcin.magnitude
        mlecapes[i] = mlecape.magnitude
    features_matching = np.where(tfm.feature_seabreeze.data == sidenum)[0]
    for matching_feat_idx in features_matching:
        matching_time_idx = tfm.feature_time_index.data[matching_feat_idx]
        feature_pressure_profile[matching_feat_idx, :] = pressure[matching_time_idx, :]
        feature_msl_profile[matching_feat_idx, :] = height[matching_time_idx, :]
        feature_temp_profile[matching_feat_idx, :] = temp[matching_time_idx, :]
        feature_dew_profile[matching_feat_idx, :] = dew[matching_time_idx, :]
        feature_u_profile[matching_feat_idx, :] = u[matching_time_idx, :]
        feature_v_profile[matching_feat_idx, :] = v[matching_time_idx, :]
        feature_ccn_profile[matching_feat_idx, :] = ccn[matching_time_idx, :]

        feature_mlcape[matching_feat_idx] = mlcapes[matching_time_idx]
        feature_mlcin[matching_feat_idx] = mlcins[matching_time_idx]
        feature_mlecape[matching_feat_idx] = mlecapes[matching_time_idx]
    
    tfm = tfm.drop_vars([f'{side}_temperature_profile', f'{side}_dewpoint_profile', f'{side}_pressure_profile', f'{side}_msl_profile',
                        f'{side}_u_profile', f'{side}_v_profile', f'{side}_ccn_profile'])

In [5]:
tfm_stats = tfm.copy()
tfm_stats = tfm_stats.assign({
    'feature_pressure_profile' : (('feature', 'vertical_levels'), feature_pressure_profile),
    'feature_msl_profile' : (('feature', 'vertical_levels'), feature_msl_profile),
    'feature_temp_profile' : (('feature', 'vertical_levels'), feature_temp_profile),
    'feature_dew_profile' : (('feature', 'vertical_levels'), feature_dew_profile),
    'feature_u_profile' : (('feature', 'vertical_levels'), feature_u_profile),
    'feature_v_profile' : (('feature', 'vertical_levels'), feature_v_profile),
    'feature_ccn_profile' : (('feature', 'vertical_levels'), feature_ccn_profile),
    'feature_mlcape' : (('feature',), feature_mlcape),
    'feature_mlcin' : (('feature',), feature_mlcin),
    'feature_mlecape' : (('feature',), feature_mlecape)
})

In [6]:
tfm_stats

<xarray.Dataset> Size: 6GB
Dimensions:                           (feature: 16210, cell: 945, x: 1001,
                                       y: 1001, time: 283, track: 770,
                                       vertical_levels: 2000)
Coordinates:
  * cell                              (cell) int64 8kB -1 4 6 ... 5092 5107 5110
  * feature                           (feature) int64 130kB 1 2 ... 16209 16210
  * time                              (time) datetime64[ns] 2kB 2022-06-02T00...
  * track                             (track) float64 6kB 0.0 1.0 ... 769.0
  * vertical_levels                   (vertical_levels) int64 16kB 0 1 ... 1999
  * x                                 (x) float64 8kB -2.5e+05 ... 2.5e+05
  * y                                 (y) float64 8kB -2.5e+05 ... 2.5e+05
Data variables: (12/73)
    avg_L2-MCMIPC                     (feature) float64 130kB ...
    cell_child_feature_count          (cell) float64 8kB ...
    cell_parent_track_id              (cell) float64 8kB ...
    feature_area                      (feature) float64 130kB ...
    feature_echotop                   (feature) int64 130kB ...
    feature_flash_count               (feature) float64 130kB ...
    ...                                ...
    feature_u_profile                 (feature, vertical_levels) float64 259MB ...
    feature_v_profile                 (feature, vertical_levels) float64 259MB ...
    feature_ccn_profile               (feature, vertical_levels) float64 259MB ...
    feature_mlcape                    (feature) float64 130kB 1.288e+03 ... 1...
    feature_mlcin                     (feature) float64 130kB -103.7 ... -103.9
    feature_mlecape                   (feature) float64 130kB 1.183e+03 ... 1...
Attributes:
    center_lat:      29.47
    center_lon:      -95.08
    soundings_used:  ['housondewnpnM1.b1.20220602.053000.cdf', 'housondewnpnM...